In [2]:
import litebird_sim as lbs
import numpy as np

start_time_s = 0
time_span_s = 1

nside = 256
npix = 12 * nside * nside

# Creatazione simulazione
sim = lbs.Simulation(
    base_path="./output",
    start_time=start_time_s,
    duration_s=time_span_s,
    random_seed=12345,
    imo=lbs.Imo(flatfile_location=lbs.PTEP_IMO_LOCATION) 
)


# Definizione della scanning strategy
sim.set_scanning_strategy(
    lbs.SpinningScanningStrategy(
        spin_sun_angle_rad=0.785_398_163_397_448_3,
        precession_rate_hz=8.664_850_513_998_931e-05,
        spin_rate_hz=0.000_833_333_333_333_333_4,
        start_time=start_time_s,
    ),
    delta_time_s=7200,
)


# Telescopio
sim.set_instrument(
    lbs.InstrumentInfo(
        boresight_rotangle_rad=0.0,
        spin_boresight_angle_rad=0.872_664_625_997_164_8,
        spin_rotangle_rad=3.141_592_653_589_793,
    ),
)


# Creazione di un oggetto detector
det = lbs.DetectorInfo(
    name="Detector",
    sampling_rate_hz=10,
    quat=[0.0, 0.0, 0.0, 1.0],
)


# Inizializzione osservazione
(obs,) = sim.create_observations(detectors=[det])


# Preparazione dei quaternioni usati per calcolare i pointings 
sim.prepare_pointings()


# Creazione di una mappa da scansionare (in una simulazione realistica usare modulo MBS fornito da litebird_sim)
maps = np.ones((3, npix))
in_map = {"Detector": maps, "Coordinates": lbs.CoordinateSystem.Ecliptic}


# Scansione della mappa per riempire un TOD
lbs.scan_map_in_observations(
    obs,
    in_map,
    input_map_in_galactic = False,
)


# Stampo a video il segnale che il detector avrebbe misurato in ogni istante di tempo durante la scansione (primi 5 campioni del TOD)
for i in range(obs.n_samples):
    value = np.round(obs.tod[0][i], 5) + 0.     # + 0. rimuove il segno meno iniziale dallo zero negativo
    print(f"{value:.5f}")

0.00000
-0.00075
-0.00151
-0.00226
-0.00301
-0.00376
-0.00451
-0.00526
-0.00601
-0.00676
